In [1]:
import numpy as np
import pandas as pd
import mplcyberpunk as mcb # if this doesn't work, pip install mplcyberpunk
import matplotlib.pyplot as plt

plt.style.use('cyberpunk') # if this 

# Import the Data

In [2]:
df = pd.read_csv("mtg_data.csv", index_col=0)
df.head()

,id,name,cmc,keywords,layout,reserved,type_line,artist,booster,border_color,...,has_blue,has_black,has_red,has_green,total_num_colors,len_oracle_text,len_mana_types_produced,has_frame_effect,has_flavor_text,class_price
0,0000579f-7b35-4ed3-b44c-db2a538066fe,Fury Sliver,6.0,[],normal,False,Creature — Sliver,Paolo Parente,True,black,...,0,0,1,0,1,40,1,0,1,0
1,0000579f-7b35-4ed3-b44c-db2a538066fe,Fury Sliver,6.0,[],normal,False,Creature — Sliver,Paolo Parente,True,black,...,0,0,1,0,1,40,1,0,1,1
2,00006596-1166-4a79-8443-ca9f82e6db4e,Kor Outfitter,2.0,[],normal,False,Creature — Kor Soldier,Kieran Yanner,True,black,...,0,0,0,0,1,118,1,0,1,0
3,00006596-1166-4a79-8443-ca9f82e6db4e,Kor Outfitter,2.0,[],normal,False,Creature — Kor Soldier,Kieran Yanner,True,black,...,0,0,0,0,1,118,1,0,1,2
4,0000cd57-91fe-411f-b798-646e965eec37,Siren Lookout,3.0,"['Flying', 'Explore']",normal,False,Creature — Siren Pirate,Chris Rallis,True,black,...,1,0,0,0,1,249,1,0,0,0


In [10]:
df.dtypes

id                          object
name                        object
cmc                        float64
keywords                    object
layout                      object
reserved                      bool
type_line                   object
artist                      object
booster                       bool
border_color                object
frame                       object
full_art                      bool
promo                         bool
rarity                     float64
released_at                 object
reprint                       bool
set                         object
story_spotlight               bool
textless                      bool
variation                     bool
power_num                  float64
toughness_num              float64
is_foil                      int64
price                      float64
has_white                    int64
has_blue                     int64
has_black                    int64
has_red                      int64
has_green           

In [20]:

len(df)/len(df['artist'].unique().tolist())

66.88113542282673

In [4]:
print(f"Number of Cards: {len(df)}")
print("Columns:")
for c in df.columns:
    print(f"\t{c}")

Number of Cards: 113096
Columns:
	id
	name
	cmc
	keywords
	layout
	reserved
	type_line
	artist
	booster
	border_color
	frame
	full_art
	promo
	rarity
	released_at
	reprint
	set
	story_spotlight
	textless
	variation
	power_num
	toughness_num
	is_foil
	price
	has_white
	has_blue
	has_black
	has_red
	has_green
	total_num_colors
	len_oracle_text
	len_mana_types_produced
	has_frame_effect
	has_flavor_text
	class_price


# Analysis

In [155]:
uq, counts = np.unique(df.class_price, return_counts=True)
perc = counts / len(df)

for i in range(len(uq)):
    print(f"{uq[i]} : {perc[i]}")

0 : 0.5524421730211502
1 : 0.1826147697531301
2 : 0.17309188653886964
3 : 0.08382259319516164
4 : 0.008028577491688477


Looking at the printout above, we see that the majority class makes up for 55% of the data. Thus we want our model to achieve an accuracy greater than 55% in order for us to have meaningful success.